## A comparison of Pythereum to Web3.py in performance, functionality and ease of use

In [1]:
import asyncio

import pythereum as pye

from eth_account import Account
from time import time
from web3 import AsyncWeb3, AsyncHTTPProvider
from ens import AsyncENS

In [2]:
# Test parameters
iterations = 1_000_000
connection_url = "127.0.0.1:8545"

In [9]:
nsum = 0
pyetimesum = 0
w3timesum = 0

w3 = AsyncWeb3(AsyncHTTPProvider(f'http://{connection_url}'))

for n in range(100, 10000, 1000):
    t0 = time()

    async with pye.EthRPC(f"ws://{connection_url}") as erpc:
        await erpc.get_block_by_number(
            list(range(n)),
            [True for i in range(n)]
        )
    print(f"pythereum processed {n} get_block_by_number requests in {(pyetime := time() - t0)} seconds")
    print(f"that's {n / pyetime} requests per second!")

    t0 = time()

    for result in asyncio.as_completed(
            [w3.eth.get_block(num, True) for num in range(n)]
        ):
          try:
              await result
          except:
              pass

    print(f"Web3.py processed {n} get_block_by_number requests in {(w3time := time() - t0)} seconds")
    print(f"that's {n / w3time} requests per second")
    
    nsum += n
    pyetimesum += pyetime
    w3timesum += w3time
    
print(f"\nOn average pythereum processed {nsum / pyetimesum} requests per second and web3.py processed {nsum / w3timesum} requests per second\nThat's a {w3timesum/pyetimesum} times speedup!")
    

pythereum processed 100 get_block_by_number requests in 0.08796906471252441 seconds
that's 1136.7632511132372 requests per second!
Web3.py processed 100 get_block_by_number requests in 0.07911086082458496 seconds
that's 1264.0489429350691 requests per second
pythereum processed 1100 get_block_by_number requests in 0.14177823066711426 seconds
that's 7758.595905902691 requests per second!
Web3.py processed 1100 get_block_by_number requests in 0.3282904624938965 seconds
that's 3350.6913105051017 requests per second
pythereum processed 2100 get_block_by_number requests in 0.2990143299102783 seconds
that's 7023.074782503307 requests per second!
Web3.py processed 2100 get_block_by_number requests in 1.4851176738739014 seconds
that's 1414.0293640989335 requests per second
pythereum processed 3100 get_block_by_number requests in 0.418917179107666 seconds
that's 7400.030733051576 requests per second!
Web3.py processed 3100 get_block_by_number requests in 1.0836989879608154 seconds
that's 2860.5

### More intensive tests, with higher iteration numbers, both processed using asyncio.gather

In [4]:
t0 = time()

# Pythereum using batch calls

async with pye.EthRPC(f"ws://{connection_url}", pool_size=10) as erpc:
    await asyncio.gather(*(
        erpc.get_block_by_number(
            [i for i in range(start, start+1000)], 
            [True for i in range(start, start+1000)]
        ) for start in range(0, iterations, 1000)
    ))
print(f"Pythereum made {iterations} requests in {(pyetime_intense := time() - t0)} seconds")

Pythereum made 1000000 requests in 134.84792470932007 seconds


In [5]:
t0 = time()
# Web3 does not natively support batch RPC calls

async def blknum(num):
    # Defined to handle errors as Web3.py does not like empty returns
    # Yes it is better to have an exception type, no it doesnt really matter
    try:
        await w3.eth.get_block(num, True)
    except:
        pass

await asyncio.gather(*(blknum(num) for num in range(iterations)))
print(f"Web3.py made {iterations} requests in {(w3time_intense := time() - t0)} seconds")

Web3.py made 1000000 requests in 377.48311281204224 seconds


In [6]:
print(f"Pythereum is able to process transactions {w3time_intense / pyetime_intense} times faster than Web3.py in intense testing,\nWith Pythereum processing {iterations / pyetime_intense} transactions per second,\nand Web3 processing {iterations / w3time_intense} transactions per second")

Pythereum is able to process transactions 2.79932459936443 times faster than Web3.py in intense testing,
With Pythereum processing 7415.7611409713045 transactions per second,
and Web3 processing 2649.1251292025972 transactions per second


### Subscriptions

In [7]:
# Pythereum subscription implementation, an infinite loop which gets block headers whenever new blocks are created

# Web3.py has no native implementation of subscriptions, and as such would have to be coded from scratch

# Items will be in the form of a Block object, with each json value converted into an appropriate data type

# Running this on localhost will result in nothing happening as no blocks are being created!

# async with pye.EthRPC(f"ws://{connection_url}") as erpc:
#     async with erpc.subscribe(pye.SubscriptionType.new_heads) as sc:
#         async for item in sc.recv():
#             print(item)

### 